In [1]:
"""
The purpose of this notebook is to write a .py callable program which takes in a local library location and returns a list of BERT tokenized word2vec vectors
"""

#need to double check if we need these if they're already in Lousia's functions
import re, string 
import pandas as pd 
from time import time  
from collections import defaultdict
import spacy
from sklearn.manifold import TSNE
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
from nltk import sent_tokenize
import matplotlib.pyplot as plt
import json
%matplotlib inline
import torch
from transformers import BertModel, BertConfig, BertTokenizer, PreTrainedTokenizer
import csv
import glob 
import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

#louisa's functions imported from local download as .py, added lines = true to fix value error when reading json into df
import Louisa_w2v_functions as w2v_functions



INFO - 11:02:39: adding document #0 to Dictionary(0 unique tokens: [])
INFO - 11:02:39: built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)


In [7]:
#write a quick function that prompts the user for a local corpus folder name then reads all the jsons in it, they then can be all cleaned w/ Louisa's clean
#function or a one I wrote

#w2v_functions.feed2vec(r"C:\Users\bowri\square1\ChemLP\Bowman\textbook_files\Brewing_Science_A_Multidisciplinary_Approach_by_Mosher_and_Trantham.json", tokenize = True )

def super_list_maker(separate = True):
    """this function prompts the user for a local folder/corpus location, where all the json's are located. It then reads all of them into a list.
    Currently it reads them into a list of lists, where each book is a list. Might be useful to have this distinction"""
    initPath = input("Enter a file location")
    bookType = input("Enter a data type(.json please)")
    token_list = []
    final_list = []
    sep_token_list_of_lists = []
    if bookType:
        path = initPath + "\*" + bookType
    else:
        path = initPath
    raw_path = r"{}".format(path)
    bookList = glob.glob(raw_path)
    
    yesOrNo = input("Is there an additional file type? (y or n) ")
    if yesOrNo == "y":
        bookType2 = input("Enter a second data type(.txt) ")
        path2 = initPath + "\*" + bookType2
        raw_path2 = r"{}".format(path2)
        bookList2 = glob.glob(raw_path2)
        for bookSite in bookList2:
            bookList.append(bookSite)
    
    bookCount = len(bookList)
    print("bookCount is ", bookCount)
    print("book list is ", bookList)

    
    for i in range(bookCount): #(len(bookList)):
        sep_book_list = w2v_functions.feed2vec(bookList[i], tokenize = True)
        sep_token_list_of_lists.append(sep_book_list[0])
        print("token list as of book ", i+1, "is ", sep_book_list[0])
        print("tokens in book ", i+1, "is", len(sep_book_list[0]))
        print()
        for word in sep_book_list[0]:
            token_list.append(word)  #use true here
        print("data type of token_list[", i+1, "]", type(token_list[i]))
        #sep_token_list_of_lists.append(token_list[i])
       # if type(token_list[i]) == list:
       #     for subtoken in token_list[i]:
       #         final_list.append(subtoken)

    print()
    print("length of vocab ", len(token_list))
    
    if separate == False:
        print("final list length/token count ", len(token_list))
        print ("all tokens ", token_list)
        return token_list
    else:
        print("list of token lists ", sep_token_list_of_lists)
        return sep_token_list_of_lists
    print()

#use a default variable here to try list of lists vs singular token list?

#local book path = C:\Users\bowri\square1\ChemLP\Bowman\textbook_files   this format is what works, then use .json as type  
#super_list_maker()


In [8]:
"""
This function needs to take in a list of tokens, set them as the global tokens value for Louisa's functions then train the model with them and return
the word embeddings
"""

#take in a vocab list

list_of_tokens = super_list_maker()
#saveName = input("Enter a name for the model saved: ")
w2v_functions.tokens = list_of_tokens
all_vecs = w2v_functions.w2v_train("fourBook", min_count = 1)




Enter a file location C:\Users\bowri\square1\ChemLP\Bowman\textbook_files
Enter a data type(.json please) .json
Is there an additional file type? (y or n)  y
Enter a second data type(.txt)  .txt


bookCount is  7
book list is  ['C:\\Users\\bowri\\square1\\ChemLP\\Bowman\\textbook_files\\Brewing_Science_A_Multidisciplinary_Approach_by_Mosher_and_Trantham.json', 'C:\\Users\\bowri\\square1\\ChemLP\\Bowman\\textbook_files\\Polymer_Chemistry_by_Koltzsenburg_Maskos_and_Nuyken.json', 'C:\\Users\\bowri\\square1\\ChemLP\\Bowman\\textbook_files\\Polymer_Synthesis_Theory_and_Practice_by_Braun_Cherdron_Rehahn_Ritter_and_Voit.json', 'C:\\Users\\bowri\\square1\\ChemLP\\Bowman\\textbook_files\\Book Basic Principles of Organic Chemistry (Roberts and Caserio).txt', 'C:\\Users\\bowri\\square1\\ChemLP\\Bowman\\textbook_files\\Book Logic of Organic Synthesis (Rao).txt', 'C:\\Users\\bowri\\square1\\ChemLP\\Bowman\\textbook_files\\Book Organic Chemistry - A Carbonyl Early Approach (McMichael).txt', 'C:\\Users\\bowri\\square1\\ChemLP\\Bowman\\textbook_files\\Soderberg_bio_o_chem.txt']
filepath is  C:\Users\bowri\square1\ChemLP\Bowman\textbook_files\Brewing_Science_A_Multidisciplinary_Approach_by_Moshe

INFO - 11:04:45: loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at C:\Users\bowri/.cache\torch\transformers\26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
WARNING - 11:04:45: Keyword arguments {'add_special_tokens': True} not recognized.


token list as of book  1 is  ['0', 'michael', 'moshe', '##r', '·', 'kenneth', 'tran', '##tham', 'brewing', 'sci', '##e', '.', '.', '.', '1', 'rights', 'reserved', 'publisher', 'whether', 'whole', 'part', 'm', '.', '.', '.', '2', 'use', 'general', 'descriptive', 'names', 'registered', 'names', '.', '.', '.', '3', 'publisher', 'authors', 'editors', 'safe', 'assume', 'advice', 'i', '.', '.', '.', '4', 'neither', 'publisher', 'authors', 'editors', 'give', 'warrant', '.', '.', '.', '.', '.', '.', '131', '##7', '##6', '131', '##7', '##7', 'k', 'g', 'k', 'j', 'k', 'p', 'r', 'n', 'e', 'g', 'k', 'j', 'k', 'p', 'l', 'h', 'n', 'e', 'g', 'k', 'j', 'k', 'g', '.', '.', '.', '131', '##7', '##8', 'b', 'e', 'l', 'b', 'appendix', 'b', 'ref', '##ri', '##ger', '##ant', 'data', 'table', 'su', '##pe', '.', '.', '.', '131', '##7', '##9', 'moshe', '##r', 'k', '131', '##80', 'tran', '##tham', 'brewing', 'science', 'multi', '##dis', '##ci', '##plin', '##ary', 'app', '.', '.', '.', 'name', ':', 'text', ',', 'len

INFO - 11:04:46: loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at C:\Users\bowri/.cache\torch\transformers\26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
WARNING - 11:04:47: Keyword arguments {'add_special_tokens': True} not recognized.


...

corpus count is  7
epochs is  30


INFO - 11:04:54: saving Word2Vec object under fourBook, separately None
INFO - 11:04:54: not storing attribute vectors_norm
INFO - 11:04:54: not storing attribute cum_table
INFO - 11:04:54: saved fourBook


keyed vectors  <gensim.models.keyedvectors.Word2VecKeyedVectors object at 0x0000027A05652308>
all_vectors(done via vocab)  

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




all_vectors type is  <class 'list'>
all words in wv.vocab are  ['0', 'michael', 'moshe', '##r', '·', 'kenneth', 'tran', '##tham', 'brewing', 'sci', '##e', '.', '1', 'rights', 'reserved', 'publisher', 'whether', 'whole', 'part', 'm', '2', 'use', 'general', 'descriptive', 'names', 'registered', '3', 'authors', 'editors', 'safe', 'assume', 'advice', 'i', '4', 'neither', 'give', 'warrant', '131', '##7', '##6', 'k', 'g', 'j', 'p', 'r', 'n', 'e', 'l', 'h', '##8', 'b', 'appendix', 'ref', '##ri', '##ger', '##ant', 'data', 'table', 'su', '##pe', '##9', '##80', 'science', 'multi', '##dis', '##ci', '##plin', '##ary', 'app', 'name', ':', 'text', ',', 'length', '81', 'dt', '##ype', 'object', 'sebastian', 'ko', '##lt', '##zen', '##burg', 'mask', '##os', 'oskar', 'nu', '##y', 'publication', 'imply', 'even', 'absence', 'specific', 'state', '##m', '950', '##4', 'see', 'blow', 'mold', '##ing', 'f', 'fenton', '’', 's', 're', '##age', '##nt', 'fiber', '##re', '##in', '##f', '##5', 'gut', '##ta', 'perch',

In [10]:
print("data type of all_vecs is ", type(all_vecs))
print(len(all_vecs))
print(all_vecs[0])

data type of all_vecs is  <class 'list'>
382
[ 5.00144670e-03 -5.85159753e-03  8.28532211e-04 -6.21038908e-03
  1.56986073e-03 -7.74002913e-03 -7.17886491e-03  6.52007898e-03
 -2.09283363e-02 -1.09173395e-02  2.25867759e-02  3.27195297e-03
 -6.99316384e-03 -1.62399318e-02 -1.48725128e-02 -1.24684153e-02
  9.29844938e-03 -9.00108460e-03 -3.93491052e-03  2.91640777e-03
 -2.15287302e-02 -1.54902483e-03  4.07102145e-03  1.18109845e-02
  1.97994011e-03 -8.05578101e-03  1.82125457e-02 -5.24422526e-03
  4.44077794e-03  7.52322190e-03  2.97036924e-04  2.30542547e-03
 -4.89422600e-05 -1.10560562e-02 -9.94485337e-03  1.47800995e-02
  1.18488083e-02 -6.06539985e-03 -1.22764912e-02 -1.79076754e-02
  4.77751996e-03  1.21954028e-02 -1.13627398e-02  8.09182297e-04
 -3.68858106e-03 -1.22039123e-02 -4.98529896e-03 -5.24759898e-03
  8.82213749e-03 -1.04142567e-02  2.04482093e-03 -2.41747759e-02
  1.27233127e-02  1.94084912e-03  2.10079588e-02 -2.06120545e-03
  1.70673162e-03  6.45194901e-04 -9.75085795e

In [5]:
"""This function/part can handle the TSNE graphing, display to user and such"""

#def grapher

'This function/part can handle the TSNE graphing, display to user and such'

In [6]:
"""
Main part of the program here. Call all the functions? Save the model/embeddings.
"""

'\nMain part of the program here. Call all the functions? Save the model/embeddings.\n'